<font color='blue'> **Lithology zone rasters to k arrays** </font> 

In [ ]:
import os; import numpy as np ; import pandas as PE ; import rasterio  ; from rasterio.plot import show  ; from pathlib import Path ; import openpyxl
TA=Path("./TA")    ; PP = Path('E:/15_REPOS/05_Working/01_nerversink') ; PB=Path(PP/'PB') ; PC=Path(PP/'PC')
PD=Path(PB/'Layers_and_Zones/K_ZONES')                                 ; method = 'mean'    ; import shutil ; import matplotlib.pyplot as plt

In [4]:
KA = PE.read_excel(PC/'Rondout_Neversink_GeologyLookupTable.xlsx'    ,sheet_name='Sheet2')  ; KA.index = KA.Lookup_Code
KB = PE.read_excel(PC/'Rondout_Neversink_GeologyLookupTable_jhw.xlsx',sheet_name='Sheet2')  ; KB.index = KB.Lookup_Code

if method == 'mean':
    KB['Kh_ft_d_mean'] = (KB['Kh_ft_d_lower'] + KB['Kh_ft_d_upper']) / 2     ; KB['Kh_m_d'] = KB['Kh_ft_d_mean']  * 0.3048
if method == 'min':                                                            KB['Kh_m_d'] = KB['Kh_ft_d_lower'] * 0.3048
if method == 'max':                                                            KB['Kh_m_d'] = KB['Kh_ft_d_lower'] * 0.3048
if method == 'original':                                                       KB = KA.copy()
KB.to_csv(TA/'_0.2_KB.csv')
compare = KB.join(KA, rsuffix='_original')               ; compare = compare[['Geology_Type', 'Kh_m_d', 'Kh_m_d_original']]
compare['diff'] = compare['Kh_m_d'] - compare['Kh_m_d_original']      ; compare.to_csv(TA/'_0.2_compare.csv')

In [5]:
def zone2k(layer,KB,new_version=True):
    global KS
    if new_version:         KS = os.path.join(PD/'L{}_K_Zone_50mGrid.tif')
    else:                   KS = os.path.join(PD/'V2_Layer{}_K.tif')          
    print('    loading: {}'.format(KS.format(layer+1)))
    
    with rasterio.open(KS.format(layer+1)) as src:
        crs = src.crs ; meta = src.meta     ; nodata = meta['nodata']   ; K_tuple = src.read(1, masked=True),   ; K_data = K_tuple[0].data.astype(float)    # , 
       
    newK = K_data.copy()
    for idx in KB.index:     newK[newK==idx] = KB.loc[idx].Kh_m_d   
    newKv = newK / 10.0      ; outfile = 'Layer{}_Kh.tif'.format(layer+1)  ; OUT_D = TA    ; OUT_P = os.path.join(OUT_D, outfile)

    DATA = rasterio.open(OUT_P, 'w', driver=meta['driver'],height = newK.shape[0], width = newK.shape[1],count=1,nodata = nodata ,dtype=str(newK.dtype),
                         crs = crs,compress='lzw',transform=meta['transform'])
    print('    writing: {}'.format(OUT_P))        ; DATA.write(newK, 1)        ; DATA.close()

    outfile = 'Layer{}_Kv.tif'.format(layer+1)    ; OUT_D = TA                 ; OUT_P = os.path.join(OUT_D, outfile)
    DATA = rasterio.open(OUT_P, 'w', driver=meta['driver'],height = newKv.shape[0], width = newKv.shape[1],count=1,nodata = nodata / 10.0 ,dtype=str(newKv.dtype),
                         crs = crs,compress='lzw',transform=meta['transform'])
    print('    writing: {}'.format(OUT_P))        ; DATA.write(newKv, 1)       ; DATA.close()

for i in range(4):
    print('now on layer {}'.format(i + 1))  ; 
    zone2k(i, KB)

now on layer 1
    loading: E:\15_REPOS\05_Working\01_nerversink\PB\Layers_and_Zones\K_ZONES\L1_K_Zone_50mGrid.tif
    writing: TA\Layer1_Kh.tif
    writing: TA\Layer1_Kv.tif
now on layer 2
    loading: E:\15_REPOS\05_Working\01_nerversink\PB\Layers_and_Zones\K_ZONES\L2_K_Zone_50mGrid.tif
    writing: TA\Layer2_Kh.tif
    writing: TA\Layer2_Kv.tif
now on layer 3
    loading: E:\15_REPOS\05_Working\01_nerversink\PB\Layers_and_Zones\K_ZONES\L3_K_Zone_50mGrid.tif
    writing: TA\Layer3_Kh.tif
    writing: TA\Layer3_Kv.tif
now on layer 4
    loading: E:\15_REPOS\05_Working\01_nerversink\PB\Layers_and_Zones\K_ZONES\L4_K_Zone_50mGrid.tif
    writing: TA\Layer4_Kh.tif
    writing: TA\Layer4_Kv.tif


In [6]:
shutil.copy(PC/'Rondout_Neversink_GeologyLookupTable.xlsx'        , TA/'_0.2_GEO_01.xlsx')      ; shutil.copy(PC/'Rondout_Neversink_GeologyLookupTable_jhw.xlsx' , TA/'_0.2_GEO_02.xlsx')
KA.to_csv(TA/'_0.2_geo_03.csv') ; KB.to_csv(TA/'_0.2_geo_04.csv') ; KS

'E:\\15_REPOS\\05_Working\\01_nerversink\\PB\\Layers_and_Zones\\K_ZONES\\L{}_K_Zone_50mGrid.tif'